In [ ]:
!pip install hmmlearn

In [39]:
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

In [48]:
##reading from the text file for observation sequnce. Only using 4.0 for now

def readInputFromFile(score):
    file_path = score
    with open(file_path, "r") as file:
        lines = file.readlines()
        print(len(lines))
        raw_input = lines[0].split("K")
        print(len(raw_input))
        hmm_input = [s.split(" ") for s in raw_input ]
    return np.array(hmm_input)

train_input = readInputFromFile("train/3.5.txt")
test_input = readInputFromFile("test/3.5.txt")

1
388


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (388,) + inhomogeneous part.

In [35]:
len(train_input)

402

In [45]:
best_score = best_model = None
n_fits = 1
N = 3
np.random.seed(13)

for idx in range(n_fits):
    model = hmm.CategoricalHMM(
        n_components=N, random_state=idx,
        init_params='se')  # don't init transition, set it below
    # we need to initialize with random transition matrix probabilities
    # because the default is an even likelihood transition
    # we know transitions are rare (otherwise the casino would get caught!)
    # so let's have an Dirichlet random prior with an alpha value of
    # (0.1, 0.9) to enforce our assumption transitions happen roughly 10%
    # of the time

    ##A 
    row_of_a = np.random.uniform(1/N - 1/(10*N),1/N + 1/(10*N),(N,N))
    A_matrix = row_of_a * N
    # print(A_matrix)
    # model.transmat_ = np.array([np.random.dirichlet([0.9, 0.1]),
    #                             np.random.dirichlet([0.1, 0.9])])
    model.transmat_ = A_matrix
    model.fit(train_input)
    score = model.score(test_input)
    print(f'Model #{idx}\tScore: {score}')
    if best_score is None or score > best_score:
        best_model = model
        best_score = score

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (388,) + inhomogeneous part.

In [47]:
array_with_sequence = np.array([np.arange(3), [1, 2, 3], [4, 5, 6]])
array_with_sequence

array([[0, 1, 2],
       [1, 2, 3],
       [4, 5, 6]])

In [27]:
# make our generative model with two components, a fair die and a
# loaded die
gen_model = hmm.CategoricalHMM(n_components=2, random_state=99)

# the first state is the fair die so let's start there so no one
# catches on right away
gen_model.startprob_ = np.array([1.0, 0.0])

# now let's say that we sneak the loaded die in:
# here, we have a 95% chance to continue using the fair die and a 5%
# chance to switch to the loaded die
# when we enter the loaded die state, we have a 90% chance of staying
# in that state and a 10% chance of leaving
gen_model.transmat_ = np.array([[0.95, 0.05],
                                [0.1, 0.9]])

# now let's set the emission means:
# the first state is a fair die with equal probabilities and the
# second is loaded by being biased toward rolling a six
gen_model.emissionprob_ = \
    np.array([[1 / 6, 1 / 6, 1 / 6, 1 / 6, 1 / 6, 1 / 6],
              [1 / 10, 1 / 10, 1 / 10, 1 / 10, 1 / 10, 1 / 2]])

# simulate the loaded dice rolls
rolls, gen_states = gen_model.sample(30000)
print(rolls)

[[2]
 [0]
 [3]
 ...
 [5]
 [5]
 [0]]
